<a href="https://colab.research.google.com/github/maxbilling/AgentspaceEvaluation/blob/main/%5BTEMPLATE%5D_AgentspaceConnectorValidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Author: @maxbilling
Based on: go/agentspace-eval-customer

Run Search Quality and Generated Answer Quality Evaluations
This notebook helps you to automate running sevaral search queries. The generated reponse to the question and top K search results are saved to a file for further review and human assessment.
Steps:

Create a csv file with columns:

*   'search_query'
*   'expected_answer'
*   'golden_url'


***'golden_url'***: Add a link to the golden response if you have one. You can also provide multiple links separated by a new line. This is for future evaluation to autogenerate metrics

Durin evaluation the following steps are performed:
5.0 search queries are imported from the file
6.1 a semantic search is performed with the default_search:search configuration of your agentspace app
6.2 based on the serach session a answer is genreated with the default_search:aswer configuraiton of your agentspace app
6.3 both results are combined the results are written back to the file

After the search and aswer results are written a evaluation takes place with

8.2. search and answer results are loaded from file
8.3. one by one evaluated by two models defined in 8.1. One model to evaluate the quality of the answer the second model to evaluate if the golden_url was part of the urls found.
8.4 the results are saved to file

# 1.0 Preparations

In [ ]:
!pip install --quiet colabtools
!pip install --quiet requests-toolbelt==1.0.0
!pip install --quiet google-cloud-discoveryengine
!pip install --upgrade --user --quiet google-cloud-aiplatform
!pip install --upgrade --quiet  langchain-core langchain-google-vertexai
!pip install --upgrade --quiet json5
!pip install --quiet scipy
!pip install --quiet gspread
!pip install --quiet --upgrade pandas pyarrow pandas-gbq
!pip install --quiet upgrade google-generativeai

# 2.0 Dependencies

In [ ]:
import json
import time
import logging
import datetime
import requests
from pprint import pprint
import pandas as pd

from google.auth import default
from google.colab import auth
import google.auth.transport.requests
import vertexai
import gspread

# Logger Setup
logger = logging.getLogger()
logging.basicConfig(filename='spark_eval_notebook_logs.log', filemode='a', level=logging.DEBUG)

creds, _ = google.auth.default()
auth_req = google.auth.transport.requests.Request()


# 3.0 Variables

In [ ]:
# Use project number and not Project ID
connector= "gcs_test" # @param {"type":"string","placeholder":"e.g. 1234567890"}
project_num = "694598770214" # @param {"type":"string","placeholder":"e.g. 1234567890"}
project = "maxbproject" # @param {"type":"string","placeholder":"e.g. 1234567890"}
# Engine ID is the ID of the Agent builder APP
engine_id = "wikipedia-agentspace_1753780981307" # @param {"type":"string","placeholder":"e.g. my-search-app_12345"}

# Assist or Search - Note: "Assist" means "Search + Assist"
# Whereas Search means "Search + Answer"
app_type = "assist" # @param ["search","assist"]

# Location
location = 'eu' # @param {"type":"string","placeholder":"Set: \"global\" "}

# evaluation region, global is not supported yet
evaluation_region = "eu" # @param {"type":"string","placeholder":"evaluation region, global is not supported yet"}
vertex_region = 'europe-west1' # @param {"type":"string","placeholder":"Set: \"global\" "}
# Use Project ID
auth_project_id = 'maxbproject' # @param {"type":"string","placeholder":"e.g. my-project-id"}

# Input Queries
# Note: Every user needs to have their own copy of this doc. Please make a copy of the golden data google sheet below and add that link.
# two columns named search_query, expected_answer
bucket = "wikipedia_source_bucket1213092" # @param {"type":"string","placeholder":"Google Sheets file URL"}
folder = "wikipedia-articles-test-2" # @param {"type":"string","placeholder":"Google Sheets file URL"}
output_file_name = 'eval_output' # @param {"type":"string","placeholder":"e.g. test_output1"}

output_dataset= 'agentspace_validation' # @param {"type":"string","placeholder":"e.g. dataset"}
# Input Queries - Note: Queries tab name within Google Sheets file
input_file_name = 'input_queries' # @param {"type":"string","placeholder":"e.g. 'input_queries'"}

# (Optional) Output file name used for debugging. This file will be saved in the colab env.

# Eval data worksheet
sheet_name_suffix = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')

do_evaluation = True # @param {"type":"boolean","placeholder":"run evaluation"}


WORKFORCE_POOL_ID="" # @param {"type":"string", "placeholder":"pool id (only if using wif)"}
WORKFORCE_PROVIDER_ID="" # @param {"type":"string", "placeholder":"provider id (only if using wif)"}
WIF_CONFIG_FILE="my_wif_auth_config.json"

# session user pseudo id
session_user_pseudo_id = "testuser01" # @param {"type":"string","placeholder":"session user pseudo id"}

# Top 'K' for search Results
K = "3" # @param {"type":"string"}

serving_config_id = "default_search" # @param {"type":"string","placeholder":"default_search"}

# 4.0 Authenticate




## 4.1 glcoud - Google User

In [107]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()


!gcloud auth application-default set-quota-project {project}


/usr/local/lib/python3.11/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)



Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "maxbproject" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


## 4.2 glcoud - Workforce Identity Federated User

In [ ]:
!gcloud iam workforce-pools create-login-config \
    locations/global/workforcePools/{WORKFORCE_POOL_ID}/providers/{WORKFORCE_PROVIDER_ID} \
    --output-file={WIF_CONFIG_FILE}
!gcloud auth login --no-launch-browser --login-config={WIF_CONFIG_FILE}
!gcloud auth application-default set-quota-project {project}


Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=HzEactqXtP8kHDlKjbbr95kVrKvVBe&prompt=consent&token_usage=remote&access_type=offline&code_challenge=MUeZotEQCwp95ERWAXwyKMW6f64nf0pqu7s7CR0qeco&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AVMBsJhplv6CvSZ6NisdykWzBeq241b5u6N4LB33ce1r5SRYwhNsLvVu3iMAFeroruKp4g

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

## 4.1 Obtain Token


In [101]:
import google.auth
import google.auth.transport.requests


def get_access_token():
  """
  Finds default credentials and returns a valid access token.
  Works for both standard users and Workforce Identity users who have already logged in.
  """
  if WORKFORCE_POOL_ID and WORKFORCE_PROVIDER_ID:
    if not access_token:
      access_token = gcloud_access_token_wif()
    return access_token

  # Find the application default credentials configured for the environment.
  creds, project_id = google.auth.default()
  # Create a transport request object to refresh the credentials.
  request = google.auth.transport.requests.Request()
  # Refresh the credentials to ensure the token is valid and not expired.
  creds.refresh(request)
  return creds.token

# --- Get and print the token ---
access_token = get_access_token()
print(f"\nAccess Token: {access_token[:20]}...") # Print first 20 chars for security


def gcloud_access_token_wif():
  ato = !gcloud auth print-access-token
  ato = list(ato)[0].strip()
  return ato




/usr/local/lib/python3.11/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)



Access Token: ya29.A0AS3H6Ny1kBBdI...


# 5.0 Import Search Queries, Expected Answers and Golden URLs from File

In [ ]:
import pandas as pd

# -----------------------------------------------------------------------------
file_name = "confluence_qa_pairs.csv"
# 1. Define the GCS path to your CSV file.
gcs_uri = f"gs://{bucket}/{folder}/{input_file_name}.csv"
print(f"GCS path: {gcs_uri}")

# 2. IMPORTANT: Define the original column names from your CSV file that you want to read.
#    Replace these placeholder names with the actual names in your file.
original_cols_to_read = ['search_query', 'expected_answer', 'golden_url']

# Define the new names for your DataFrame columns.
new_column_names = {
    'search_query': 'search_query',
    'expected_answer': 'expected_answer',
    'golden_url': 'golden_url'
}

# 3. Read only the specified columns from GCS and rename them in one step.
print(f"Reading data from GCS bucket: {gcs_uri}")
golden_data = pd.read_csv(
    gcs_uri,
    usecols=original_cols_to_read  # Only load these specific columns
).rename(columns=new_column_names) # Rename them to your desired names

# 4. Display the first 10 rows and columns to confirm it loaded correctly.
print("\n--- DataFrame Info ---")
golden_data.info()

print("\n--- DataFrame Head ---")
print(golden_data.head(2))

GCS path: gs://wikipedia_source_bucket1213092/wikipedia-articles-test-2/input_queries.csv
Reading data from GCS bucket: gs://wikipedia_source_bucket1213092/wikipedia-articles-test-2/input_queries.csv

--- DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190 entries, 0 to 189
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   search_query     190 non-null    object
 1   expected_answer  190 non-null    object
 2   golden_url       190 non-null    object
dtypes: object(3)
memory usage: 4.6+ KB

--- DataFrame Head ---
                                        search_query  \
0  Which seeded player from South Korea successfu...   
1  In the second qualifier's draw, what was the f...   

                                     expected_answer  \
0                Cho Yoon-jeong, who was seeded 5th.   
1  Colombia Fabiola Zuluaga defeated Tunisia Seli...   

                                          gol

# 6.0 Helper Functions

## 6.1 Get Search Results

In [ ]:
import requests
import logging
import subprocess

# Configure a basic logger to see output
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


def get_search_results(query, num=K):
    try:

        access_token = get_access_token()

        url = f"https://{evaluation_region}-discoveryengine.googleapis.com/v1alpha/projects/{project_num}/locations/{location}/collections/default_collection/engines/{engine_id}/servingConfigs/{serving_config_id}:search"


        headers = {
            "Authorization": f"Bearer {access_token}",
            "Content-Type": "application/json"
        }

        data = {
            "servingConfig": f"projects/{project_num}/locations/{location}/collections/default_collection/engines/{engine_id}/servingConfigs/{serving_config_id}",
            "query": query,
            "pageSize": num,
            "session":f"projects/{project_num}/locations/{location}/collections/default_collection/engines/{engine_id}/sessions/-"
        }

        search_response = requests.post(url, headers=headers, json=data)

        if search_response.status_code == 200:
            try:
                return search_response.json()#['results'][0]['document']['derivedStructData']['link']
            except requests.exceptions.JSONDecodeError:
                logger.error(f"Query '{query}' returned status 200 but response was not valid JSON. Body: {search_response.text}")
                return {}
        else:
            logger.error(
                f"API call for query '{query}' failed with status {search_response.status_code}: {search_response.reason}\n"
                f"Response Body: {search_response.text}"
            )
            return {}

    # THIS IS THE FIX: The 'except' block completes the 'try' statement
    except (subprocess.CalledProcessError, requests.exceptions.RequestException, NameError) as e:
        logger.error(f"An error occurred during the search process: {e}")
        # Return an empty dictionary to allow the main loop to continue safely
        return {}

## 6.2 Get Answer Results

In [ ]:


import requests
import logging

# Assume logger and other variables (creds_token, evaluation_region, etc.) are defined

def get_answer_results(query, queryId, session_name):
    try:
        access_token = get_access_token()

        url = f"https://{evaluation_region}-discoveryengine.googleapis.com/v1alpha/projects/{project_num}/locations/{location}/collections/default_collection/engines/{engine_id}/servingConfigs/{serving_config_id}:answer"

        headers = {
            "Authorization": f"Bearer {access_token}",
            "Content-Type": "application/json"
        }

        # The data dictionary with corrected Python syntax
        data = {
            "servingConfig": f"projects/{project_num}/locations/{location}/collections/default_collection/engines/{engine_id}/servingConfigs/{serving_config_id}",
            "query": {"text": query, "queryId": queryId},
            "session": session_name,
            "relatedQuestionsSpec": {"enable": True},
            "answerGenerationSpec": {
                "ignoreAdversarialQuery": True,
                "ignoreNonAnswerSeekingQuery": False,
                "ignoreLowRelevantContent": True,
                "multimodalSpec": {},
                "includeCitations": True
            }
        }

        search_response = requests.post(url, headers=headers, json=data)

        if search_response.status_code == 200:
            try:
                # Returns the full JSON response if successful
                return search_response.json()
            except requests.exceptions.JSONDecodeError:
                logger.error(f"Query '{query}' returned status 200 but response was not valid JSON. Body: {search_response.text}")
                return {}
        else:
            logger.error(
                f"API call for query '{query}' failed with status {search_response.status_code}: {search_response.reason}\n"
                f"Response Body: {search_response.text}"
            )
            return {}

    except (requests.exceptions.RequestException, NameError) as e:
        logger.error(f"An error occurred during the search process: {e}")
        # Return an empty dictionary to allow the main loop to continue safely
        return {}


## 6.3 Get Full Results

In [ ]:
import json
import datetime

# Assuming get_search_results() and get_answer_results() are defined correctly

def get_full_response_dict(query: str, K: int) -> dict:
    """
    Processes a query to get search and answer results and returns them in a dictionary.
    """
    try:
        # --- Step 1: Get session info from a search call ---
        search_data = get_search_results(query, K)
        session_info = search_data.get('sessionInfo', {})
        session_name = session_info.get('name')
        queryId = session_info.get('queryId')


        #print(get_search_results(query, K))

        # --- Step 2: Get the generative answer ---
        if queryId and session_name:
            answer_data = get_answer_results(query, queryId, session_name)
            #print (get_answer_results(query, queryId, session_name))
        else:
            print("Warning: Could not find session_name or queryId to get an answer.")
            answer_data = {}

        # --- Step 3: Extract all data points and build the dictionary ---
        if answer_data:
            # Safely get the nested 'answerText'
            answer_details = answer_data.get('answer', {})
            answer_text = answer_details.get('answerText')

            answerQueryToken = answer_data.get('answerQueryToken')
            attributionToken = search_data.get('attributionToken')

            # Extract all result links
            response_urls = []
            relevance_scores = []
            structured_search_results = []
            for result in search_data.get('results', []):
                link_struct={}
                link = result.get('document', {}).get('derivedStructData', {}).get('link')
                relevance_score = result.get('modelScores', {}).get('relevance_score',{}).get('values',[])[0]


                if link:
                    link_struct = {
                        "link": link,
                        "relevance_score": relevance_score
                    }
                    structured_search_results.append(link_struct)


            # Create the final dictionary and return it
            response_dict = {
                'query': query,
                'answer': answer_text,
                'attributionToken': attributionToken,
                'answerQueryToken': answerQueryToken,
                'searchResults': structured_search_results

            }
            return response_dict

        else:
            print("Failed to get a valid answer from the API.")
            # Return an empty dictionary on failure
            return {}

    except Exception as e:
        print(f"An error occurred: {e}")
        # Return an empty dictionary on error
        return {}


# 7.0 Main Processing Loop - Get Answers with Top K Search Results

In [ ]:
import pandas as pd
import numpy as np
import datetime
from google.cloud import storage
from tqdm import tqdm

# --- Configuration ---
BATCH_SIZE = 20

# --- 1. Define a single Evaluation Run ID at the beginning ---
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
assist_run_id = f"{connector}_{timestamp}"
print(f"🚀 Starting new run. Search & Assist Run with ID: {assist_run_id}")

# --- Define GCS paths ---
TEMP_GCS_FOLDER = f"{folder}/temp_batches_{assist_run_id}" # Temp folder is now unique to the run
TEMP_GCS_DIR_URI = f"gs://{bucket}/{TEMP_GCS_FOLDER}"
# -----------------------------------------------------------------------------

def process_row(row):
    """
    Function to process a single row of the DataFrame.
    (This function remains unchanged)
    """
    query = row['search_query']
    api_results_dict = get_full_response_dict(query, K)
    output_data = row.to_dict()
    output_data.update(api_results_dict)
    return output_data

# -----------------------------------------------------------------------------
# Main Processing Loop
# -----------------------------------------------------------------------------
print("\n--- Starting Search Query Processing in Batches (GCS) ---")

individual_gcs_paths = []
tqdm.pandas(desc="Processing queries")

# 2. Split the DataFrame into batches
list_of_batches = np.array_split(golden_data, len(golden_data) // BATCH_SIZE + 1)
total_batches = len(list_of_batches)
print(f"Processing {len(golden_data)} rows in {total_batches} batches...")

# 3. Loop through and process each batch
for i, batch_df in enumerate(list_of_batches):
    current_batch_num = i + 1
    print(f"\n--- Processing Batch {current_batch_num}/{total_batches} ---")

    if batch_df.empty:
        print("Skipping empty batch.")
        continue

    # A. Apply the processing function
    results_list = batch_df.progress_apply(process_row, axis=1).tolist()
    processed_batch_df = pd.DataFrame(results_list)

    # B. Add the consistent eval_run_id to the batch DataFrame
    processed_batch_df['assist_run_id'] = assist_run_id

    # C. Write batch results to a temporary CSV file in GCS
    batch_gcs_uri = f"{TEMP_GCS_DIR_URI}/batch_{current_batch_num}.csv"
    print(f"Saving batch results to: {batch_gcs_uri}")
    processed_batch_df.to_csv(batch_gcs_uri, index=False)
    individual_gcs_paths.append(batch_gcs_uri)
    print(f"✅ Batch {current_batch_num} saved to GCS.")
    #add a delay in order not to hit rate limits
    time.sleep(1)

# 4. --- Merge, Save Final File, and Clean Up GCS ---
print("\n--- Finalizing Results from GCS ---")

if not individual_gcs_paths:
    print("Warning: No batch files were created. Final DataFrame is empty.")
else:
    # A. Merge all individual CSVs directly from GCS
    print(f"Merging {len(individual_gcs_paths)} temporary CSV files from GCS...")
    evaluation_df = pd.concat(
        (pd.read_csv(gcs_uri) for gcs_uri in individual_gcs_paths),
        ignore_index=True
    )

    # B. Save the final merged DataFrame to its final destination in GCS
    # The output filename now includes the run ID for better tracking
    final_output_file_name = f"{output_file_name}_{assist_run_id}.csv"
    final_gcs_uri = f"gs://{bucket}/{folder}/{final_output_file_name}"

    print(f"\nSaving final merged results to GCS path: {final_gcs_uri}...")
    evaluation_df.to_csv(final_gcs_uri, index=False)
    print(f"✅ Successfully saved final results to GCS: {final_gcs_uri}")

    # C. Clean up by deleting the temporary files from GCS
    print(f"\nCleaning up temporary files from {TEMP_GCS_DIR_URI}...")
    try:
        storage_client = storage.Client(project=project)
        bucket = storage_client.bucket(bucket)
        blobs_to_delete = list(bucket.list_blobs(prefix=TEMP_GCS_FOLDER))

        if blobs_to_delete:
            bucket.delete_blobs(blobs_to_delete)
            print(f"✅ Successfully deleted {len(blobs_to_delete)} temporary files from GCS.")
        else:
            print("No temporary files found to delete.")

    except Exception as e:
        print(f"Error during GCS cleanup: {e}")

print("\n--- Cloud-based batch processing complete. ---")

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


🚀 Starting new run. Evaluation Run ID: gcs_test_20250813_111354

--- Starting Search Query Processing in Batches (GCS) ---
Processing 190 rows in 10 batches...

--- Processing Batch 1/10 ---


Processing queries:  68%|██████▊   | 13/19 [01:43<00:51,  8.56s/it]

An error occurred: list index out of range


Processing queries: 100%|██████████| 19/19 [02:30<00:00,  8.02s/it]

...

An error occurred: list index out of range


Processing queries:  89%|████████▉ | 17/19 [01:56<00:15,  7.82s/it]

An error occurred: list index out of range


Processing queries: 100%|██████████| 19/19 [02:20<00:00,  7.39s/it]


Saving batch results to: gs://wikipedia_source_bucket1213092/wikipedia-articles-test-2/temp_batches_gcs_test_20250813_111354/batch_10.csv
✅ Batch 10 saved to GCS.

--- Finalizing Results from GCS ---
Merging 10 temporary CSV files from GCS...

Saving final merged results to GCS path: gs://wikipedia_source_bucket1213092/wikipedia-articles-test-2/eval_output_gcs_test_20250813_111354.csv...
✅ Successfully saved final results to GCS: gs://wikipedia_source_bucket1213092/wikipedia-articles-test-2/eval_output_gcs_test_20250813_111354.csv

Cleaning up temporary files from gs://wikipedia_source_bucket1213092/wikipedia-articles-test-2/temp_batches_gcs_test_20250813_111354...
✅ Successfully deleted 10 temporary files from GCS.

--- Cloud-based batch processing complete. ---


# 8.0 Answer Evaluation


## 8.1. Defintion of Evaluation Model

In [113]:
import pandas as pd
import json
import time
from google import genai
import traceback


# This is the prompt to validate the quality of the answer results and rate it on a scale
EVALUATION_PROMPT = """
You are a meticulous and objective expert evaluator. Your task is to compare a generated `Answer` to a ground-truth `Expected Answer`, strictly considering the context of the original `Query`. Your evaluation must determine if the `Expected Answer` is factually and semantically contained within the generated `Answer`.

You must rate the match on the scale of 0 to 5 defined below. Your response must be ONLY a single, valid JSON object with two keys: "rating" (an integer from 0-5) and "justification" (a brief string, no more than 1-2 sentences, explaining your rating based on the scale). Do not add any text before or after the JSON object.

---
[RATING SCALE DEFINITION]

- **5 (Excellent Match):** The `Answer` perfectly contains all the information from the `Expected Answer`. It is factually correct, complete, and concise, with no significant missing or extraneous information.

- **4 (Very Good Match):** The `Answer` contains all the key information from the `Expected Answer` but may include minor, harmless additional details or have slightly different phrasing that doesn't alter the meaning.

- **3 (Good/Partial Match):** The `Answer` contains the main point of the `Expected Answer` but is missing some important secondary details, or it contains a notable amount of extra, less relevant information.

- **2 (Fair/Minor Match):** The `Answer` contains some correct elements from the `Expected Answer` but misses the primary point or is factually incomplete in a significant way.

- **1 (Poor Match):** The `Answer` touches on the topic of the `Query` but provides information that is mostly irrelevant or fails to align with the `Expected Answer`.

- **0 (No Match):** The `Answer` contains no relevant or correct information from the `Expected Answer`. It is completely off-topic, factually wrong, or a refusal to answer.
---

[TASK]
Query: "{query}"
Expected Answer: "{expected_answer}"
Answer: "{answer}"

Your Response:
"""

# This is the prompt to validate if the golden link is found in the search link results
LINK_PROMPT ="""
Your one and only task is to respond with either TRUE or FALSE, and nothing else.

You are a meticulous and objective expert evaluator. Your task is to compare a generated `searchResults` to a ground-truth `goldenLink`. The 'goldenLink' contains a page ID. Try to find if this page ID is present within the search results.

The links do not have to match exactly, but the document name must match. Your final response must be the word TRUE or FALSE, and nothing else.


---
[TASK]
goldenLink: "{goldenLink}"
searchResults: "{searchResults}"
"""


# Initialize the Model
client = genai.Client(vertexai=True, project=project, location=vertex_region)
MODEL_ID = "gemini-2.5-flash"



def evaluate_answer(row):
    """
    Calls Gemini to evaluate an answer.
    """
    try:
        answer = str(row['answer'])
        query = str(row['search_query'])
        expected_answer = str(row['expected_answer'])

        prompt = EVALUATION_PROMPT.format(
            query=query,
            expected_answer=expected_answer,
            answer=answer
        )
        # The GenerationConfig class is accessed from `genai` now

                # Instead of a genai.GenerationConfig object, use a simple dictionary.
        # 1. Use a dictionary for the configuration.
        generation_config_dict = {
            "response_mime_type": "application/json",
            "temperature": 0
        }

        # 2. Use your original, correct method call, passing the dictionary to the 'config' parameter.
        response = client.models.generate_content(
            model=MODEL_ID,
            contents=prompt,
            config=generation_config_dict
        )

        # This check is crucial to handle blocked responses
        if not response.candidates:
            justification = f"Error: Model returned an empty or blocked response. Reason: {response.prompt_feedback}"
            print(justification) # Print error for debugging
            return pd.Series({'rating': -1, 'justification': justification})

        result_dict = json.loads(response.text)

        # This is the successful return path
        return pd.Series({
            'rating': int(result_dict.get('rating', -1)),
            'justification': str(result_dict.get('justification', ''))
        })
    except Exception as e:
        # This general except block catches ALL other errors and ensures a Series is returned
        error_message = f'An unexpected error occurred: {e}'
        print(error_message) # Print error for debugging
        return pd.Series({'rating': -1, 'justification': error_message})


def evaluate_link(row):
    """
    Calls Gemini to evaluate links and returns the full error if one occurs.
    """
    try:
        goldenLink = str(row['golden_url'])
        search_results_list = str(row['searchResults'])

        prompt = LINK_PROMPT.format(
            goldenLink=goldenLink,
            searchResults=search_results_list,
        )
        generation_config_dict = {
            "response_mime_type": "application/json",
            "temperature": 0
        }

        # 2. Use your original, correct method call, passing the dictionary to the 'config' parameter.
        response = client.models.generate_content(
            model=MODEL_ID,
            contents=prompt,
            config=generation_config_dict
        )

        return pd.Series({
            'link_evaluation': response.text,
        })

    except Exception as e:
        print("--- An error occurred. Capturing full traceback. ---")
        full_error_details = traceback.format_exc()
        print(full_error_details)
        return pd.Series({
            'link_evaluation': None
        })


def evaluate_row_fully(row):


    """
    A wrapper function that calls both evaluate_answer and evaluate_link
    for a single row and combines their results into one Series.
    """
    # Call the first evaluation function
    answer_result = evaluate_answer(row)

    # Call the second evaluation function
    link_result = evaluate_link(row)

    # Rename the outputs to prevent column name collisions.
    answer_result = answer_result.rename({
        'rating': 'answer_rating',
        'justification': 'answer_justification'
    })

    # Combine the results from both functions into a single pandas Series
    full_result = pd.concat([answer_result, link_result])

    return full_result

## 8.2 Load the Evaluation Data from File

In [109]:
import pandas as pd
from tqdm import tqdm
import time

evaluation_df =[]
#Uncomment this line, if a specific file should be used for the evaluation
#final_output_file_name = 'eval_output_gcs_test_20250813_111354'


# 2. Construct the full GCS URI for the output file.
input_file = f"{final_output_file_name}.csv"
input_gcs_uri = f"gs://{bucket}/{folder}/{input_file}"

try:
    evaluation_df =  pd.read_csv(
    input_gcs_uri
)
    print(f"Successfully loaded {input_gcs_uri}.")
except FileNotFoundError:
    print("Error: 'search_eval.csv' not found. Please ensure the file from the previous step exists.")
    # Create a dummy dataframe for demonstration if file not found
    evaluation_df = pd.DataFrame({
        'answer': ["Gemini is a family of multimodal models.", "The sky is blue due to Rayleigh scattering."],
        'expected_answer': ["Gemini is a family of models.", "The sky appears blue because of how the atmosphere interacts with sunlight."]
    })
#print(evaluation_df)

Successfully loaded gs://wikipedia_source_bucket1213092/wikipedia-articles-test-2/eval_output_gcs_test_20250813_111354.csv.


## 8.3 Test the evluation with a single row



In [110]:
import pandas as pd
import json
import google.generativeai as genai
import google.auth

from google.colab import userdata # <-- Import userdata to access secrets


# --- 2. Prepare the Data for the SDK ---
test_data = {
    'search_query': ["what is Gemini?"],
    'expected_answer': ["Gemini is magic"],
    'answer': ["Gemini is a AI model made by Google"],
    'golden_url' : [["https://somerandomlink.com/type/1234/thegoldendocument.csv"]],
    'searchResults' : [[
        {'source': 'gs://wikipedia_source_bucket1213092/wikipedia-articles/2002 US Open  Womens singles qualifying_a96b60.pdf'},
        {'source': 'gs://wikipedia_source_bucket1213092/wikipedia-articles/thegoldendocument.csv'},
        {'source': 'gs://wikipedia_source_bucket1213092/wikipedia-articles/2022 European Speed Skating Championships  Womens team sprint_abd5c1.pdf'}
    ]]
}
test_df = pd.DataFrame(test_data)

print("\n--- Running evaluation on the single test row ---")
test_row = test_df.iloc[0]

# --- 3. Create and Run the Evaluation Task ---
print("\n--- Defining and running the evaluation task ---")


evaluation_result = evaluate_row_fully(test_row)

# --- 4. Display the Results ---
print("\n--- Evaluation Result ---")
# The result object has a convenient method to convert it to a DataFrame

print(evaluation_result)



--- Running evaluation on the single test row ---

--- Defining and running the evaluation task ---

--- Evaluation Result ---
answer_rating                                                           0
answer_justification    The generated answer states Gemini is an AI mo...
link_evaluation                                                    "TRUE"
dtype: object


## 8.4 Run the evaluation and save output to file

In [ ]:
import pandas as pd
import numpy as np
import datetime
import os
from tqdm import tqdm

# --- Configuration ---
BATCH_SIZE = 20
TEMP_BATCH_DIR = "temp_evaluation_batches" # Local temporary directory for batch CSVs


# define the eval run id
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
eval_run_id = f"{connector}_{timestamp}"

# --- Main Evaluation Loop ---
print("\n--- Starting AI-based evaluation of answers in batches ---")

# Ensure the necessary column exists before proceeding
if 'expected_answer' not in evaluation_df.columns:
    print("Error: 'expected_answer' column not found. Stopping evaluation.")
else:
    # 1. Setup for batch processing
    os.makedirs(TEMP_BATCH_DIR, exist_ok=True) # Create temp dir for batch files
    individual_csv_paths = [] # To store paths of batch CSVs for final merge
    tqdm.pandas(desc="Evaluating answer quality")

    # Split the DataFrame into batches
    list_of_batches = np.array_split(evaluation_df, len(evaluation_df) // BATCH_SIZE + 1)
    total_batches = len(list_of_batches)

    print(f"Processing {len(evaluation_df)} rows in {total_batches} batches of up to {BATCH_SIZE} rows each.")

    # 2. Loop through each batch
    for i, batch_df in enumerate(list_of_batches):
        current_batch_num = i + 1
        print(f"\n--- Processing Batch {current_batch_num}/{total_batches} ---")

        if batch_df.empty:
            print("Skipping empty batch.")
            continue

        # A. Apply the evaluation function to the current batch
        # Using .progress_apply for a progress bar within the batch
        batch_results = batch_df.progress_apply(evaluate_row_fully, axis=1)

        if isinstance(batch_results, pd.Series):
            batch_results = batch_results.to_frame().T

        # B. Join results back to the batch DataFrame
        # Define columns that might already exist and need to be replaced
        cols_to_replace = ['answer_rating', 'answer_justification', 'link_evaluation']
        existing_cols_to_drop = [col for col in cols_to_replace if col in batch_df.columns]

        if existing_cols_to_drop:
            batch_df = batch_df.drop(columns=existing_cols_to_drop)

        processed_batch_df = batch_df.join(batch_results)

        # C. Add metadata
        processed_batch_df['connector'] = connector
        processed_batch_df['evaluation_timestamp'] = eval_run_id

        # D. Write individual batch results to a local CSV file
        batch_file_path = os.path.join(TEMP_BATCH_DIR, f"batch_{current_batch_num}.csv")
        processed_batch_df.to_csv(batch_file_path, index=False)
        individual_csv_paths.append(batch_file_path)
        print(f"✅ Batch {current_batch_num} saved locally to: {batch_file_path}")

        # E. Append results to BigQuery
        table_name = f"{connector}_validation"
        destination_table = f"{output_dataset}.{table_name}"

        print(f"Appending batch {current_batch_num} to BigQuery table: {project}.{destination_table}...")
        processed_batch_df.to_gbq(
            destination_table=destination_table,
            project_id=project,
            if_exists='append', # Append results for each batch
            progress_bar=True
        )
        print(f"✅ Successfully appended batch {current_batch_num} to BigQuery.")

    # 3. --- Post-Loop: Merge individual CSVs and Clean Up ---
    print("\n--- Finalizing Results ---")

    # A. Merge all individual CSVs into one DataFrame
    if not individual_csv_paths:
        print("Warning: No batch CSV files were created to merge.")
    else:
        print(f"Merging {len(individual_csv_paths)} individual CSV files...")
        merged_df = pd.concat((pd.read_csv(f) for f in individual_csv_paths), ignore_index=True)

        # B. Save the final merged CSV to Google Cloud Storage
        output_file = f"{output_file_name}.csv"
        output_gcs_uri = f"gs://{bucket}/{folder}/{output_file}"

        print(f"Saving final merged CSV to GCS path: {output_gcs_uri}...")
        merged_df.to_csv(output_gcs_uri, index=False)
        print(f"✅ Successfully saved merged results to GCS: {output_gcs_uri}")

        # C. Clean up by deleting the local temporary files and directory
        print("Cleaning up temporary batch files...")
        for file_path in individual_csv_paths:
            try:
                os.remove(file_path)
            except OSError as e:
                print(f"Error deleting file {file_path}: {e}")
        try:
            os.rmdir(TEMP_BATCH_DIR)
            print(f"✅ Successfully removed temporary directory: {TEMP_BATCH_DIR}")
        except OSError as e:
            print(f"Error removing directory {TEMP_BATCH_DIR}: {e}")

print("\n--- Evaluation process complete. ---")

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)



--- Starting AI-based evaluation of answers in batches ---
Processing 190 rows in 10 batches of up to 20 rows each.

--- Processing Batch 1/10 ---


Evaluating answer quality:  47%|████▋     | 9/19 [01:18<01:30,  9.06s/it]